In [163]:
import CreateSamples
import OT2_Ouzo_Commands_API2c as ot2_commands
import Ouzo_Graph_Tools as ouzo_graphs
from opentrons import simulate, execute, protocol_api
import os
import pandas as pd
import glob, json
import numpy as np

In [164]:
import importlib
importlib.reload(CreateSamples)

<module 'CreateSamples' from 'C:\\Users\\Edwin\\Desktop\\OT2Protocols\\ot2protocol\\Ouzo_OT2_Sampling\\CreateSamples.py'>

In [173]:
# these variables although have the same names when searching for the stocks should now replace those for the actual selectede one. 
path = r"C:\Users\Edwin\Desktop\OT2Protocols\ot2protocol\Ouzo_OT2_Sampling\Testing Plans\Example_Working_Protocol.csv"
experiment_csv_dict = CreateSamples.get_experiment_plan(path)
wtf_sample_canidates = CreateSamples.generate_candidate_lattice_concentrations(experiment_csv_dict)
volume_sample_canidates = CreateSamples.calculate_ouzo_volumes_from_wtf(wtf_sample_canidates, experiment_csv_dict)
final_volume_df = CreateSamples.pipette_volume_restriction_df(volume_sample_canidates, 30, 1000, experiment_csv_dict['Pipette Entire Stock Volume Only Once?'])

In [434]:
df = final_volume_df*1.25

In [445]:
volumes = np.asarray(df['ethanol-PFH-stock uL'])
limit = 18000

def ranges(array, limit):
    volume_limited = array.cumsum() % limit
    checks = [i<=j for i, j in zip(volume_limited, volume_limited[1:])]
    if True in (all(checks), all(v == 0 for v in volume_limited)):
        return [[0,len(volume_limited)]]
    
    # from here is frustration, very confusing but currently works
    stops = [0]
    for i, tf in enumerate(checks): 
        if tf == False:
            stops.append(i+1)

    if stops[-1] == len(volume_limited): # essentially if a stop at the end 
        pass
    else: 
        stops.append(stops[-1])
        stops.append(len(volume_limited)) # adding final index 

    n = 2
    ranges = [stops[i:i+n] for i in range(0, len(stops), n)]
    return ranges
#     for range_ in ranges: 
#         print(volume_limited[range_[0]:range_[1]])

x = ranges(volumes, limit)


## Step 5: Run the OT2 commands
* Now that a stock and its corresponding volhave been selected run the command code.
* Future: make it so the can edit stock information and print the amount of stock needed (instructions for stock creation)

In [446]:
labware_dir_path = r"C:\Users\Edwin\Desktop\OT2Protocols\ot2protocol\Ouzo_OT2_Sampling\Custom Labware"
labware_dict = ot2_commands.custom_labware_dict(labware_dir_path)
custom_labware_dict= labware_dict

In [447]:
# now need to simimulate first or actually grab the stock labware
# you will need to have the stock labware you are going to use in the the dict as well to grab the max values aND the amount of slots
# then with this information you can calcualte how many stocks you need or if you should switch/add to another labware/ if does not fit
# To keep it easy just keep it so stock 1 is made in order so if you need say 40mL it is in A1 and A2 and then stock 2s would start in A3

In [448]:
# pipetting in order of components, where each component has a seperate stock
# grab the volumes [1,2,3]
# intialize a well with a certain stock, so starting with A1
# sum until reach a value, 2
# IF you reach the value before the end, record the index range of A1, then make a new entry for A2 and record the indexes aagin
# Then start with the next component and run through the whole thing again


# make a stock df to see placements, which will later be used for along with the volume inputs.

In [466]:
protocol = simulate.get_protocol_api('2.0', extra_labware=labware_dict)
df = final_volume_df * 1.25
experiment_dict = experiment_csv_dict
stock_plate = ot2_commands.custom_or_native_labware(protocol, experiment_dict['OT2 Stock Labware'], experiment_dict['OT2 Stock Labware Slot'], custom_labware_dict)
stock_plate_rows = [well for row in stock_plate.rows() for well in row]
stock_plate_well_volume = stock_plate.__dict__['_well_definition']['A1']['totalLiquidVolume'] 
limit = stock_plate_well_volume
final_ranges = []
for col in df: 
    if 'stock' in col:
        stock_volumes = np.array(df[col])
        stock_ranges = ranges(stock_volumes, limit)
        final_ranges.append((stock_ranges))

# final_ranges = np.asarray(final_ranges)
(final_ranges)
# df2 = pd.DataFrame([[0],[1],[2],[3],[4]],
#                    columns=df.columns[:-1])


C:\Users\Edwin\.opentrons\deck_calibration.json not found. Loading defaults
C:\Users\Edwin\.opentrons\robot_settings.json not found. Loading defaults


[[[0, 91]], [[0, 91]], [[0, 91]], [[0, 67], [67, 91]], [[0, 54], [54, 91]]]

In [457]:
df.columns[:-1]

Index(['dppc-stock uL', 'dspe-peg-stock uL', 'ethanol-PFH-stock uL',
       'ethanol-stock uL', 'water-stock uL'],
      dtype='object')

In [425]:
protocol = simulate.get_protocol_api('2.0', extra_labware=labware_dict)
command_info = ot2_commands.create_samples(protocol, experiment_csv_dict, volumes) 



C:\Users\Edwin\.opentrons\deck_calibration.json not found. Loading defaults
C:\Users\Edwin\.opentrons\robot_settings.json not found. Loading defaults


TypeError: object of type 'numpy.float64' has no len()

In [ ]:
# in order to change the dispense and aspiration speed - to match the gen1 (), use command: pipette.flow_rate.aspirate/dispense

# when not enough samples present also make sure to raise errors for things like not inputting enough slots/ len of labware does not equal len of slots 

In [ ]:
protocol1 = simulate.get_protocol_api('2.0')
ot2_commands.simple_independent_transfer(protocol1, dict_1['experiment_plan_dict'])

## Step 6: Create a csv file containing all the information of the experiment for storage and future linking

In [ ]:
create.create_csv(r"C:\Users\Edwin\Desktop\test", command_info['command info'], seleted_wtfs_plus_blank, experiment_csv_dict)
df = pd.read_csv(r"C:\Users\Edwin\Desktop\test")
# make it upload to team drive on google and have it so you can pull from there as well. 

In [ ]:
df